In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

from tensorflow.keras import layers

### DataLoader

In [2]:
def get_mnist_dataset():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    
    x_train = x_train.reshape(60000, 784).astype('float32') / 255
    x_test = x_test.reshape(10000, 784).astype('float32') / 255
    
    y_train = y_train.astype('float32')
    y_test = y_test.astype('float32')
    
    x_val = x_train[-10000:]
    y_val = y_train[-10000:]
    x_train = x_train[:-10000]
    y_train = y_train[:-10000]
    
    return x_train, y_train, x_val, y_val, x_test, y_test

In [4]:
x_train, y_train, x_val, y_val, x_test, y_test = get_mnist_dataset()

## Model with Functional API

In [32]:
def get_simple_mlp():
    tf.random.set_seed(42)  # set random seed
    
    inputs = tf.keras.Input(shape=(784,), name='digits')
    x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
    x = layers.Dense(32, activation='relu', name='dense_2')(x)
    outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [33]:
def model_compiler(
    model,
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['sparse_categorical_accuracy']):
    
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=metrics)
    return model

In [37]:
model_1 = get_simple_mlp()
model_1 = model_compiler(model_1)

model_2 = get_simple_mlp()
model_2 = model_compiler(model_2)

In [38]:
model_1.fit(x_train, y_train, batch_size=64, epochs=3)

Train on 50000 samples
Epoch 1/3
50000/50000 [==============================] - 3s 64us/sample - loss: 1.7218 - sparse_categorical_accuracy: 0.7545
Epoch 2/3
50000/50000 [==============================] - 3s 69us/sample - loss: 1.6188 - sparse_categorical_accuracy: 0.8467
Epoch 3/3
50000/50000 [==============================] - 5s 105us/sample - loss: 1.6047 - sparse_categorical_accuracy: 0.8584


In [39]:
model_2.fit(x_train, y_train, batch_size=64, epochs=3)

Train on 50000 samples
Epoch 1/3
50000/50000 [==============================] - 3s 59us/sample - loss: 1.7218 - sparse_categorical_accuracy: 0.7545
Epoch 2/3
50000/50000 [==============================] - 3s 66us/sample - loss: 1.6188 - sparse_categorical_accuracy: 0.8467
Epoch 3/3
50000/50000 [==============================] - 4s 80us/sample - loss: 1.6047 - sparse_categorical_accuracy: 0.8584


### Checking determinism

In [78]:
def check_determinism(model_1, model_2):
    
    print('Check trainable weights')  # check trainable weights
    weights_1 = model_1.trainable_weights
    weights_2 = model_2.trainable_weights
    for w1, w2 in zip(weights_1, weights_2):
        w1 = w1.numpy().flatten()
        w2 = w2.numpy().flatten()
        print('num params: ', len(w1))
        np.testing.assert_allclose(w1, w2, rtol=1e-6, atol=1e-6)
        
    print('\nCheck validation scores')  # check validation socres
    val_1 = model_1.evaluate(x_val, y_val, batch_size=64, verbose=3)
    val_2 = model_2.evaluate(x_val, y_val, batch_size=64, verbose=3)
    print('model_1 - loss: {:.8f}, metric: {}'.format(val_1[0], val_1[1:]))
    print('model_2 - loss: {:.8f}, metric: {}'.format(val_2[0], val_2[1:]))
    
    print('\nCheck test predictions')  # check test predictions
    pred_1 = model_1.predict(x_test)
    pred_2 = model_2.predict(x_test)
    print('num preds: ', len(pred_1))
    np.testing.assert_allclose(pred_1, pred_2, rtol=1e-6, atol=1e-6)

In [79]:
check_determinism(model_1, model_2)

Check trainable weights
num params:  50176
num params:  64
num params:  2048
num params:  32
num params:  320
num params:  10

Check validation scores
model_1 - loss: 1.59803915, metric: [0.8634]
model_2 - loss: 1.59803915, metric: [0.8634]

Check test predictions
num preds:  10000


## Model with Subclassing

In [137]:
class MLP(tf.keras.Model):
    
    def __init__(self):
#         tf.random.set_seed(42)
        super(MLP, self).__init__()
        self.dense_1 = layers.Dense(64, activation='relu', name='dense_1')
        self.dense_2 = layers.Dense(32, activation='relu', name='dense_2')
        self.classifier = layers.Dense(10, activation='softmax', name='predictions')
        
    def call(self, inputs):
#         tf.random.set_seed(42)
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        return self.classifier(x)

In [138]:
# tf.random.set_seed(42)
mlp_1 = MLP()
# tf.random.set_seed(42)
mlp_2 = MLP()

In [139]:
# tf.random.set_seed(42)
mlp_1 = model_compiler(mlp_1)
# tf.random.set_seed(42)
mlp_2 = model_compiler(mlp_2)

In [140]:
tf.random.set_seed(42)
mlp_1.fit(x_train, y_train, batch_size=64, epochs=3)

Train on 50000 samples
Epoch 1/3
50000/50000 [==============================] - 3s 61us/sample - loss: 1.7218 - sparse_categorical_accuracy: 0.7545
Epoch 2/3
50000/50000 [==============================] - 2s 49us/sample - loss: 1.6188 - sparse_categorical_accuracy: 0.8467
Epoch 3/3
50000/50000 [==============================] - 2s 50us/sample - loss: 1.6047 - sparse_categorical_accuracy: 0.85840s - loss: 1.6051 - sparse_categorical_accuracy: 0.


In [141]:
tf.random.set_seed(42)
mlp_2.fit(x_train, y_train, batch_size=64, epochs=3)

Train on 50000 samples
Epoch 1/3
50000/50000 [==============================] - 4s 75us/sample - loss: 1.7218 - sparse_categorical_accuracy: 0.7545
Epoch 2/3
50000/50000 [==============================] - 2s 47us/sample - loss: 1.6188 - sparse_categorical_accuracy: 0.8467
Epoch 3/3
50000/50000 [==============================] - 3s 53us/sample - loss: 1.6047 - sparse_categorical_accuracy: 0.8584


In [144]:
check_determinism(mlp_1, mlp_2)

Check trainable weights
num params:  50176
num params:  64
num params:  2048
num params:  32
num params:  320
num params:  10

Check validation scores
model_1 - loss: 1.59803915, metric: [0.8634]
model_2 - loss: 1.59803915, metric: [0.8634]

Check test predictions
num preds:  10000


when only `set random seed` before `fit` call, determinism works